In [2]:
import imageio_ffmpeg
import shapely

import geopandas as gpd
import matplotlib as mpl
import matplotlib.patches as mpatch
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox

from pathlib import Path
from pyfonts import load_font

In [121]:
BACKGROUND_COLOR = "#75d4ff"
EDGE_COLOR = "#00598d"
TEXT_COLOR = "#00a7f7"
FPS = 12

In [3]:
centroid_path = Path(r"C:\Users\lain\OneDrive - Instituto Tecnologico y de Estudios Superiores de Monterrey\centroids\historical")
reprojected_path = Path(r"C:\Users\lain\OneDrive - Instituto Tecnologico y de Estudios Superiores de Monterrey\population_grids_data\final\reprojected\merged")

In [ ]:
font = load_font(font_path="./fonts/AvenirNextLTPro-Bold.otf")

In [123]:
def generate_figure():
    fig, (axl, axr) = plt.subplots(1, 2, figsize=(16, 9), width_ratios=(2, 1))
    return fig, axl, axr


def generate_circle_patch(center: tuple[float, float], width: float, height: float, *, alpha: float=1, color: str=BACKGROUND_COLOR) -> mpatch.Ellipse:
    return mpatch.Ellipse(
        xy=center,
        width=width,
        height=height,
        color=color,
        alpha=alpha
    )

def annotate_axis(ax: mpl.axes.Axes, name: str, *, alpha: float=1):
    ax.annotate(
        name,
        xy=(0.5, 0.6),
        xycoords="axes fraction",
        ha="center",
        va="center",
        fontsize=32,
        color=TEXT_COLOR,
        font=font,
        alpha=alpha
    )
    ax.axis("off")


def plot_with_background(subg: nx.Graph | None, *, ax, xmin:float, xmax: float, ymin: float, ymax: float):
    if subg is not None:
        ox.plot.plot_graph(subg, node_size=0, ax=ax, edge_color=EDGE_COLOR, show=False)
 
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect("equal")


logo = plt.imread("./figures/CFC2.jpg")
def add_logo(ax):
    axin = ax.inset_axes([0.05, 0.5, 0.9, 0.9])
    axin.imshow(logo)
    axin.axis("off")

In [119]:
def plot_graph(g: nx.Graph, bfs_edges: list, *, frame_offset: int=0, name: str, xmax: float, xmin: float, ymax: float, ymin: float, title_fade_in_frames: int, center: tuple[float, float]) -> None:
    for i, edges in enumerate(bfs_edges):
        subg = g.edge_subgraph(edges)

        fig, axl, axr = generate_figure()
        plot_with_background(subg, ax=axl, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

        alpha = np.min((1, (i + 1) / title_fade_in_frames))
        annotate_axis(axr, name, alpha=alpha)

        add_logo(axr)

        fig.savefig(f"./frames/{(i + frame_offset):06d}.png", dpi=300, bbox_inches="tight")
        plt.close()


def erase_frames(g: nx.Graph, last_edges: list, *, frame_offset: int, fade_out_frames: int, hold_frames: int, empty_frames: int, name: str, xmin: float, xmax: float, ymin: float, ymax: float, center: tuple[float, float]) -> None:
    width = xmax - xmin
    height = ymax - ymin

    subg = g.edge_subgraph(last_edges)

    for i in range(hold_frames):
        fig, axl, axr = generate_figure()
        plot_with_background(subg, ax=axl, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
        annotate_axis(axr, name)

        add_logo(axr)

        fig.savefig(f"./frames/{(i + frame_offset):06d}.png", dpi=300, bbox_inches="tight")
        plt.close()

    for i in range(fade_out_frames):
        fig, axl, axr = generate_figure()
        plot_with_background(subg, ax=axl, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

        circle_patch = generate_circle_patch(center, width+0.0005, height+0.0005, alpha=(i + 1) / fade_out_frames, color="#FFFFFF")
        axl.add_patch(circle_patch)
        
        annotate_axis(axr, name, alpha=(1 - (i + 1) / fade_out_frames))

        add_logo(axr)

        fig.savefig(f"./frames/{(i + frame_offset + hold_frames):06d}.png", dpi=300, bbox_inches="tight")
        plt.close()

    for i in range(empty_frames):
        fig, axl, axr = generate_figure()
        
        plot_with_background(subg, ax=axl, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
        circle_patch = generate_circle_patch(center, width+0.0005, height+0.0005, alpha=1, color="#FFFFFF")
        axl.add_patch(circle_patch)

        annotate_axis(axr, name, alpha=0)

        add_logo(axr)
        
        fig.savefig(f"./frames/{(i + frame_offset + hold_frames + fade_out_frames):06d}.png", dpi=300, bbox_inches="tight")
        plt.close()

In [126]:
offset = 0

wanted_zones = {
    "19.1.01": "Monterrey",
    "09.1.01": "Ciudad de\nMéxico",
    "02.2.03": "Mexicali",
    "25.2.01": "Culiacán",
    "14.1.01": "Guadalajara",
    "08.2.03": "Ciudad Juárez",
}

for zone, name in wanted_zones.items():
    centroid = gpd.read_file(centroid_path / f"{zone}.gpkg").to_crs("EPSG:4326")["geometry"]
    circle = (
        centroid
        .to_crs("EPSG:6372")
        .item()
        .buffer(3_000, resolution=32)
    )
    circle = gpd.GeoSeries([circle], crs="EPSG:6372").to_crs("EPSG:4326").item()

    xmin, ymin, xmax, ymax = circle.bounds

    g = ox.graph_from_polygon(circle)
    g = ox.convert.to_undirected(g)

    nodes, edges = ox.convert.graph_to_gdfs(g)

    closest_node = int(nodes.to_crs("EPSG:6372").distance(centroid.to_crs("EPSG:6372").item()).sort_values().index[0])

    bfs_edges = []

    depth_limit = 1
    while True:
        next_edges = list(nx.bfs_edges(g, closest_node, depth_limit=depth_limit))
        if len(bfs_edges) > 0 and len(bfs_edges[-1]) == len(next_edges):
            break

        next_edges_extended = [(edge[0], edge[1], 0) for edge in next_edges]
        bfs_edges.append(next_edges_extended)
        depth_limit += 1

    center = tuple(circle.centroid.coords[0])

    plot_graph(
        g, 
        bfs_edges, 
        frame_offset=offset, 
        name=name, 
        xmax=xmax, 
        xmin=xmin, 
        ymax=ymax, 
        ymin=ymin, 
        title_fade_in_frames=int(np.round(1.5 * FPS)),
        center=center
    )
    erase_frames(
        g, 
        bfs_edges[-1], 
        frame_offset=len(bfs_edges) + offset, 
        hold_frames=2 * FPS, 
        fade_out_frames=int(np.round(1.5 * FPS)),
        empty_frames=FPS,
        name=name, 
        xmin=xmin, 
        xmax=xmax, 
        ymin=ymin, 
        ymax=ymax,
        center=center
    )

    offset += len(bfs_edges) + 2 * FPS + int(np.round(1.5 * FPS)) + FPS